# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [40]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

#Configure gmaps
gmaps.configure(api_key=g_key)



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [41]:
# Output File (CSV)
input_data_file = "../output_data/cities.csv"

#Read WeatherPy CSV file
Final_city_data= pd.read_csv(input_data_file)

Final_city_data.head()

,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Poya,-21.35,165.15,84.61,72,100,2.75,NC,1609728176
1,Pevek,69.70,170.31,-18.15,89,72,6.80,RU,1609728178
2,New Norfolk,-42.78,147.06,69.01,52,93,3.00,AU,1609728179
3,Kapaa,22.08,-159.32,78.80,65,90,13.87,US,1609728180
4,Mount Isa,-20.73,139.50,91.40,41,75,6.93,AU,1609728181


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [42]:
#Use the Lat and Lng as locations and Humidity as the weight.
locations = Final_city_data[["Latitude", "Longitude"]].astype(float)

humidity= Final_city_data["Humidity"].astype(float)


In [43]:
fig= gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig = gmaps.figure(center=(0, 0), zoom_level=2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [44]:
#Narrow down the cities to fit weather conditions.
Perfect_humidity= Final_city_data.loc[Final_city_data["Humidity"] <= 68]

Perfect_temp= Perfect_humidity.loc[Perfect_humidity["Max Temperature"] > 75 ]

#Store into variable named hotel_df.
hotel_df= Perfect_temp.loc[Perfect_temp["Cloudiness"] <= 30 ]

hotel_df.head()

,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
9,Kahului,20.89,-156.47,80.60,54,1,24.16,US,1609728186
42,San Lorenzo,-25.33,-57.53,82.40,58,0,5.82,PY,1609728165
89,Hilo,19.73,-155.09,80.60,61,1,9.17,US,1609728209
230,Broken Hill,-31.95,141.43,78.80,39,29,10.29,AU,1609728445
356,Trat,12.50,102.50,84.99,65,0,11.41,TH,1609728593


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
#Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

hotel_df.head()

,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
9,Kahului,20.89,-156.47,80.60,54,1,24.16,US,1609728186,
42,San Lorenzo,-25.33,-57.53,82.40,58,0,5.82,PY,1609728165,
89,Hilo,19.73,-155.09,80.60,61,1,9.17,US,1609728209,
230,Broken Hill,-31.95,141.43,78.80,39,29,10.29,AU,1609728445,
356,Trat,12.50,102.50,84.99,65,0,11.41,TH,1609728593,


In [46]:
# Hit the Google Places API for each city's coordinates.
target_search = "Hotel"
target_radius = 5000


#Set parameters to search for specifics
params = { 
    "keyword": target_search,
    "radius": target_radius, 
    "key":g_key, 
}

In [47]:
print("Beginning Data Retrieval")
print("-------------------------")

# Use iterrows to iterate through dataframe
for index, row in hotel_df.iterrows():

    #Store latitude and longitude from hotel_df
    lat = row['Latitude']
    lng = row['Longitude']


    #Update location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"


    #Set the base_url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_names = requests.get(base_url, params=params)
    hotel_names = hotel_names.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_names["results"][0]["name"]

    except (KeyError, IndexError, TypeError):
        print("Missing result. Skipping... .")

Beginning Data Retrieval
-------------------------
Missing result. Skipping... .
Missing result. Skipping... .
Missing result. Skipping... .


In [48]:
pprint(hotel_names)

a'},
             {'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 4.864049, 'lng': 31.60039},
                           'viewport': {'northeast': {'lat': 4.865518079892722,
                                                      'lng': 31.60174142989272},
                                        'southwest': {'lat': 4.862818420107279,
                                                      'lng': 31.59904177010727}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'Airport Plaza Hotel',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 2448,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/115545295131029702001">Airport '
                                                'Plaza Hotel</a>'],
                          'photo_reference': 'ATtYBwIgFzhCV5WbU

In [49]:
# Save Data to csv file
hotel_df.to_csv("../output_data/Perfect_weather_hotels.csv", index= False, header= True)

# Read CSV 
Perfect_weather_hotels= pd.read_csv("../output_data/Perfect_weather_hotels.csv")

Perfect_weather_hotels.head()

,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Kahului,20.89,-156.47,80.60,54,1,24.16,US,1609728186,Maui Beach Hotel
1,San Lorenzo,-25.33,-57.53,82.40,58,0,5.82,PY,1609728165,Factoria Hotel
2,Hilo,19.73,-155.09,80.60,61,1,9.17,US,1609728209,Hilo Hawaiian Hotel
3,Broken Hill,-31.95,141.43,78.80,39,29,10.29,AU,1609728445,Royal Exchange Hotel
4,Trat,12.50,102.50,84.99,65,0,11.41,TH,1609728593,NaN


In [50]:
Renamed_df = Perfect_weather_hotels.rename(columns= {"City Name" : "City"})

Renamed_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Kahului,20.89,-156.47,80.60,54,1,24.16,US,1609728186,Maui Beach Hotel
1,San Lorenzo,-25.33,-57.53,82.40,58,0,5.82,PY,1609728165,Factoria Hotel
2,Hilo,19.73,-155.09,80.60,61,1,9.17,US,1609728209,Hilo Hawaiian Hotel
3,Broken Hill,-31.95,141.43,78.80,39,29,10.29,AU,1609728445,Royal Exchange Hotel
4,Trat,12.50,102.50,84.99,65,0,11.41,TH,1609728593,NaN
5,Panlaitan,12.12,119.85,84.58,66,9,5.77,PH,1609728608,NaN
6,Papetoai,-17.50,-149.87,86.00,55,20,2.24,PF,1609728616,Hilton Moorea Lagoon Resort & Spa
7,‘Āhuimanu,21.44,-157.84,78.80,65,20,13.87,US,1609728645,NaN
8,Kampot,10.62,104.18,82.40,58,0,6.93,KH,1609728654,Hotel Five.S
9,Juba,4.85,31.58,76.24,49,19,2.46,SS,1609728687,Pyramid Continental Hotel


Plot markers on top of the heatmap.

In [58]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in Renamed_df.iterrows()]
locations = Renamed_df[["Latitude", "Longitude"]]


In [65]:
# Add marker layer 

marker_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig= gmaps.figure()

#Zoom in before adding layers
fig = gmaps.figure(center=(0, 0), zoom_level=2)

#Add marker layer
fig.add_layer(marker_layer)

#Add heat layer
fig.add_layer(heat_layer)

# fig = gmaps.figure(center=(0, 0), zoom_level=2)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))